In [118]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import copy, math
from sklearn.ensemble import RandomForestClassifier

In [119]:
metadata=pd.read_csv(r"E:\BT workshop\PTB\metadata\metadata.csv")
relabd_species=pd.read_csv(r"E:\BT workshop\PTB\taxonomy\taxonomy_relabd.species.csv")
metadata.index=[i for i in metadata['specimen']]


In [120]:
relabd_species.index=[i for i in relabd_species['specimen']]
relabd_species.drop('specimen',inplace=True,axis=1)
relabd_species[relabd_species<0.001]=0


In [121]:
relabd_species['was_preterm']=metadata['was_preterm']
relabd_species['collect_wk']=metadata['collect_wk']
relabd_species['trimester'] = relabd_species['collect_wk'].apply(lambda x: 1 if x/12 <= 1 else (2 if x/12 <= 2 else 3))

In [122]:
relabd_species.head()

,'Geomonas' Khan et al. 2020,Abditibacterium,Abiotrophia defectiva,Absiella,Absiella dolichum,Abyssicoccus,Acetobacter,Acetobacter cibinongensis/orientalis,Acetobacter persici,Acetobacteraceae,...,unclassified Clostridiales,unclassified Gammaproteobacteria,unclassified Lachnospiraceae,unclassified Rhodospirillales,unclassified Ruminococcaceae,unclassified Tissierellia,unclassified Xanthomonadaceae,was_preterm,collect_wk,trimester
A00001-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,33,3
A00002-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,38,3
A00003-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,30,3
A00004-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,27,3
A00004-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,29,3


In [123]:
grouped=relabd_species.groupby('was_preterm')

In [124]:
term=grouped.get_group(False)
preterm=grouped.get_group(True)

In [125]:
preterm

,'Geomonas' Khan et al. 2020,Abditibacterium,Abiotrophia defectiva,Absiella,Absiella dolichum,Abyssicoccus,Acetobacter,Acetobacter cibinongensis/orientalis,Acetobacter persici,Acetobacteraceae,...,unclassified Clostridiales,unclassified Gammaproteobacteria,unclassified Lachnospiraceae,unclassified Rhodospirillales,unclassified Ruminococcaceae,unclassified Tissierellia,unclassified Xanthomonadaceae,was_preterm,collect_wk,trimester
A00007-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,34,3
A00008-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,17,2
A00008-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,26,3
A00008-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,30,3
A00008-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,31,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J00086-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,18,2
J00087-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,16,2
J00088-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,17,2
J00090-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,17,2


In [126]:
preterm.to_csv('preterm.csv', index=False)
term.to_csv('term.csv', index=False)

In [127]:
preterm.head()

,'Geomonas' Khan et al. 2020,Abditibacterium,Abiotrophia defectiva,Absiella,Absiella dolichum,Abyssicoccus,Acetobacter,Acetobacter cibinongensis/orientalis,Acetobacter persici,Acetobacteraceae,...,unclassified Clostridiales,unclassified Gammaproteobacteria,unclassified Lachnospiraceae,unclassified Rhodospirillales,unclassified Ruminococcaceae,unclassified Tissierellia,unclassified Xanthomonadaceae,was_preterm,collect_wk,trimester
A00007-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,34,3
A00008-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,17,2
A00008-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,26,3
A00008-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,30,3
A00008-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,31,3


In [128]:
preterm_groups = preterm.groupby([preterm.index.str[0],'trimester'])
term_groups = term.groupby([term.index.str[0],'trimester'])

In [129]:
preterm.drop('was_preterm',inplace=True,axis=1)

term.drop('was_preterm',inplace=True,axis=1)
preterm.drop('collect_wk',inplace=True,axis=1)

term.drop('collect_wk',inplace=True,axis=1)
preterm.drop('trimester',inplace=True,axis=1)

term.drop('trimester',inplace=True,axis=1)



C:\Users\THAZEEL\AppData\Local\Temp\ipykernel_9980\543506138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preterm.drop('was_preterm',inplace=True,axis=1)
C:\Users\THAZEEL\AppData\Local\Temp\ipykernel_9980\543506138.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  term.drop('was_preterm',inplace=True,axis=1)
C:\Users\THAZEEL\AppData\Local\Temp\ipykernel_9980\543506138.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preterm

In [130]:
print(*preterm_groups)

(('A', 1),            'Geomonas' Khan et al. 2020  Abditibacterium  \
A00012-01                          0.0              0.0   
A00022-03                          0.0              0.0   
A00022-05                          0.0              0.0   

           Abiotrophia defectiva  Absiella  Absiella dolichum  Abyssicoccus  \
A00012-01                    0.0       0.0                0.0           0.0   
A00022-03                    0.0       0.0                0.0           0.0   
A00022-05                    0.0       0.0                0.0           0.0   

           Acetobacter  Acetobacter cibinongensis/orientalis  \
A00012-01          0.0                                   0.0   
A00022-03          0.0                                   0.0   
A00022-05          0.0                                   0.0   

           Acetobacter persici  Acetobacteraceae  ...  \
A00012-01                  0.0               0.0  ...   
A00022-03                  0.0               0.0  ...   
A00022-

In [131]:
result_preterm={}
for name, group in preterm_groups:
    group_rows = group[group > 0] 
    col_mask = (group_rows.count() / len(group_rows)) >= 0.5 
    cols = col_mask[col_mask].index.tolist() 
    if cols: 
        letter, trimester = name
        if letter not in result_preterm:
            result_preterm[letter] = {}
        result_preterm[letter][trimester] = cols
result_term={}
for name, group in term_groups:
    group_rows = group[group > 0] 
    col_mask = (group_rows.count() / len(group_rows)) >= 0.5 
    cols = col_mask[col_mask].index.tolist() 
    if cols: 
        letter, trimester = name
        if letter not in result_term:
            result_term[letter] = {}
        result_term[letter][trimester] = cols

In [132]:
result_preterm

{'A': {1: ['Atopobium vaginae',
   'Bacteroides fragilis',
   'Bacteroides vulgatus',
   'Dialister',
   'Faucicola',
   'Gardnerella vaginalis',
   'Haemophilus haemolyticus',
   'Lachnospira eligens',
   'Megamonas',
   'Prevotella',
   'Prevotella timonensis',
   'Streptococcus salivarius',
   'Veillonella'],
  2: ['Bacteroides fragilis',
   'Bacteroides vulgatus',
   'Faucicola',
   'Gardnerella vaginalis',
   'Haemophilus haemolyticus',
   'Megamonas',
   'Prevotella',
   'Prevotella disiens',
   'Prevotella timonensis',
   'Prevotellaceae',
   'Veillonella'],
  3: ['Alistipes onderdonkii',
   'Bacteroides fragilis',
   'Bacteroides vulgatus',
   'Dialister',
   'Faucicola',
   'Fenollaria massiliensis/timonensis',
   'Gardnerella vaginalis',
   'Lactobacillus iners',
   'Neisseria oralis',
   'Prevotella colorans',
   'Prevotella disiens',
   'Prevotella timonensis',
   'Prevotellaceae',
   'Ruminococcaceae',
   'Ureaplasma parvum',
   'Veillonella']},
 'B': {1: ['Lactobacillus c

In [133]:
len(result_preterm)

10

In [134]:
# for key, inner_dict in list(result_preterm.items()):
#     for inner_key, inner_value in list(inner_dict.items()):
#         if len(inner_value) < 10:
#             del result_preterm[key][inner_key]
#     if not result_preterm[key]:
#         del result_preterm[key]


In [135]:
result_preterm

{'A': {1: ['Atopobium vaginae',
   'Bacteroides fragilis',
   'Bacteroides vulgatus',
   'Dialister',
   'Faucicola',
   'Gardnerella vaginalis',
   'Haemophilus haemolyticus',
   'Lachnospira eligens',
   'Megamonas',
   'Prevotella',
   'Prevotella timonensis',
   'Streptococcus salivarius',
   'Veillonella'],
  2: ['Bacteroides fragilis',
   'Bacteroides vulgatus',
   'Faucicola',
   'Gardnerella vaginalis',
   'Haemophilus haemolyticus',
   'Megamonas',
   'Prevotella',
   'Prevotella disiens',
   'Prevotella timonensis',
   'Prevotellaceae',
   'Veillonella'],
  3: ['Alistipes onderdonkii',
   'Bacteroides fragilis',
   'Bacteroides vulgatus',
   'Dialister',
   'Faucicola',
   'Fenollaria massiliensis/timonensis',
   'Gardnerella vaginalis',
   'Lactobacillus iners',
   'Neisseria oralis',
   'Prevotella colorans',
   'Prevotella disiens',
   'Prevotella timonensis',
   'Prevotellaceae',
   'Ruminococcaceae',
   'Ureaplasma parvum',
   'Veillonella']},
 'B': {1: ['Lactobacillus c